In [5]:

import pandas as pd
import numpy as np


In [6]:
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 


Data downloaded successfully.


In [1]:
# Unzipping the data
import zipfile
with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
    zip_ref.extractall("competition_data")

In [13]:
#!pip install tensorflow_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 7.7 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=ebf2168deb5277db9bcdd8a5572522d6dd526b239efda4095a33907c80fb21ec
  Stored in directory: /Users/rishabhsalwan/Library/Caches/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

In [7]:
def preprocessor(imageband_directory):
    import PIL
    import os
    import numpy as np
    import tensorflow_datasets as tfds
    
    def _load_tif(data):
        img = tfds.core.lazy_imports.PIL_Image.open(data)    
        # loads data from a single tif image in the form of an arrayof size (400,)
        img = np.array(img.getdata()).reshape(img.size).astype(np.float32)    
        # Transforms the image array to a matrix of 20x20 and convert each value to float type    
        return img
    
    image_list = []
    filelist1 = os.listdir(imageband_directory)
    # gets list of all image files in the folder
    
    for fpath in filelist1:
    # loop over all the image names in the given folder
        name = imageband_directory+"/"+fpath
        # this is the complete path name for a given image
        
        if fpath.endswith(("B05.tif","B06.tif","B07.tif","B11.tif","B12.tif","B8A.tif")):
        # if the image is in the red, blue or green band execute the following code
            image_array = _load_tif(name)
            # image array is the 120x120 matrix for the image (either red, green or blue)
            image_list.append(image_array)
            # For each image we get a list of 120x120x3 matrices
        if fpath.endswith(("B02.tif","B03.tif","B04.tif","B08.tif")):
            x = _load_tif(name)
            y = []
            for i in range(119):
                if i%2==0:
                    row = []
                    for j in range(119):
                        if j%2==0:
                            row.append(np.mean(np.array([x[i,j],x[i,j+1],x[i+1,j],x[i+1,j+1]])))
                    y.append(row)
                    
            image_list.append(np.array(y))

    
    X = np.stack(image_list, axis = 2)
    # Converts the list into an multidimensional array of shape 120x120x3 i.e. each image has been stacked on top of another
    X = np.expand_dims(X, axis = 0)
    # Adds an extra dimension to convert the above shape in the form 1x20x20x3. Probably for keras model input
    X = np.array(X, dtype=np.float32)
    # Convert all elements to float
    X = X/18581
    # Normalising the data for better ML algorithm performance and reduce bias
    X = X.flatten()
    
    return X

In [8]:
forestfilenames=["competition_data/trainingdata/forest/"+x for x in os.listdir("competition_data/trainingdata/forest")]
# Getting all forest subfolder paths
nonforestfilenames=["competition_data/trainingdata/nonforest/"+x for x in os.listdir("competition_data/trainingdata/nonforest")]
# Getting a list of all non-forest sub folder paths
otherfilenames=["competition_data/trainingdata/other/"+x for x in os.listdir("competition_data/trainingdata/other")]
# Getting a list of other sub folder paths

filenames=forestfilenames+nonforestfilenames+otherfilenames
# Combining all the lists into one; all subfolder paths in one place

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in tqdm(filenames):
# i is one subfolder path i.e folder of images(image directory)
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass
# loop returns a list of 120x120x3 arrays as indvidual elements, each element corresponds to a folder

100%|██████████| 15007/15007 [1:24:42<00:00,  2.95it/s]   


In [10]:
60*60*10

36000

In [9]:
np.shape(preprocessed_image_data[0])

(36000,)

In [11]:
# First 5000 images are forest, then non-forest and lastly other
from itertools import repeat
forest=repeat("forest",5000)
nonforest=repeat("nonforest",5000)
other=repeat("snow_shadow_cloud",5000)
ylist=list(forest)+list(nonforest)+list(other)
# combining to get a list of y's

In [12]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(preprocessed_image_data, ylist, random_state=0)
# X_train has all the folder arrays shuffled in the form of a list; has 15000 elements (image folders) with shape(120,120,3)
# And their corresponding y values accordingly shuffled in the form of a list; has length (15000)

In [13]:
X_train=np.vstack(X_train) # convert X from list to array
# vertically stacking X list elements in the form of an array

In [14]:
X_train.shape

(15000, 36000)

In [15]:
# Simple Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
model = RandomForestClassifier(n_estimators = 100, max_depth = 10, random_state=0)
model.fit(X_train,y_train)
prediction_labels = model.predict(X_train)

f1_score(y_train, prediction_labels, average="macro")

0.8513721811291443

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
importances = model.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)),feature_names)
plt.yticks(range(len(indices)),np.array(feature_names)[indices])
plt.xlabel('Relative Importance')

KeyboardInterrupt: 

In [16]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"")

cannot pickle 'module' object
Your preprocessor is now saved to 'preprocessor.zip'


In [66]:
#!brew install libomp

Running `brew update --preinstall`...
==> Homebrew is run entirely by unpaid volunteers. Please consider donating:
  https://github.com/Homebrew/brew#donations
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> Updated Formulae
Updated 1 formula.

==> Downloading https://ghcr.io/v2/homebrew/core/libomp/manifests/13.0.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libomp/blobs/sha256:c751c4d5205
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Pouring libomp--13.0.1.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/libomp/13.0.1: 9 files, 1.6MB
==> Running `brew cleanup libomp`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


In [68]:
#!pip install pydot

In [70]:
#!brew install graphviz

==> Downloading https://ghcr.io/v2/homebrew/core/libpng/manifests/1.6.37
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libpng/blobs/sha256:7209cfe63b2
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/freetype/manifests/2.11.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/freetype/blobs/sha256:e1320cb3e
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/fontconfig/manifests/2.13.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/fo

==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/mpdecimal/manifests/2.5.1
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/mpdecimal/blobs/sha256:73e9acc9
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/ca-certificates/manifests/2022-
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/ca-certificates/blobs/sha256:40
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/home

==> Pouring libpng--1.6.37.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/libpng/1.6.37: 27 files, 1.3MB
==> Installing graphviz dependency: freetype
==> Pouring freetype--2.11.1.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/freetype/2.11.1: 66 files, 2.3MB
==> Installing graphviz dependency: fontconfig
==> Pouring fontconfig--2.13.1.monterey.bottle.tar.gz
==> Regenerating font cache, this may take a while
==> /usr/local/Cellar/fontconfig/2.13.1/bin/fc-cache -frv
🍺  /usr/local/Cellar/fontconfig/2.13.1: 530 files, 3.6MB
==> Installing graphviz dependency: jpeg
==> Pouring jpeg--9e.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/jpeg/9e: 21 files, 939.4KB
==> Installing graphviz dependency: brotli
==> Pouring brotli--1.0.9.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/brotli/1.0.9: 25 files, 2.3MB
==> Installing graphviz dependency: giflib
==> Pouring giflib--5.2.1.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/giflib/5.2.1: 19 files, 583.7KB
==> Installing graphviz dependency: imath
==> Pouring imath

==> Pouring cairo--1.16.0_5.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/cairo/1.16.0_5: 126 files, 6.3MB
==> Installing graphviz dependency: gdk-pixbuf
==> Pouring gdk-pixbuf--2.42.6.monterey.bottle.tar.gz
==> /usr/local/Cellar/gdk-pixbuf/2.42.6/bin/gdk-pixbuf-query-loaders --update-ca
🍺  /usr/local/Cellar/gdk-pixbuf/2.42.6: 147 files, 3.7MB
==> Installing graphviz dependency: fribidi
==> Pouring fribidi--1.0.11.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/fribidi/1.0.11: 67 files, 525.8KB
==> Installing graphviz dependency: gobject-introspection
==> Pouring gobject-introspection--1.70.0_3.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/gobject-introspection/1.70.0_3: 191 files, 13.0MB
==> Installing graphviz dependency: graphite2
==> Pouring graphite2--1.3.14.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/graphite2/1.3.14: 18 files, 276KB
==> Installing graphviz dependency: icu4c
==> Pouring icu4c--70.1.monterey.bottle.tar.gz
🍺  /usr/local/Cellar/icu4c/70.1: 261 files, 74.4MB
==> Installing g

In [17]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=X_train.shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  #Insert correct number of preprocessed features

onnx_model = model_to_onnx(model, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [18]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers=[str(x) for x in range(1, 5001)]
filenames=["competition_data/testdata/test/test"+x for x in filenumbers]

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in filenames:
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass  

In [19]:
X_test=np.vstack(preprocessed_image_data) 

In [20]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:········
AI Modelshare Password:········
AI Model Share login credentials set successfully.


In [21]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [22]:
#Submit Model: 

#-- Generate predicted values (a list of predicted image categories) 
predicted_values = model.predict(X_test)

# Submit model to Competition Leaderboard

custom_meta = {'team': 'KLSRS',
               'model_type': 'Random_Forest',
               'Member1': 'Krutarth Saraswath',
              'Member2': 'Rishabh Salwan'}

mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=predicted_values,
                                 custom_metadata = custom_meta)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 159

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [23]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

In [ ]:
# Custom metadata can be added by passing a dict to the custom_metadata argument of the submit_model() method
# This option can be used to fill in missing data points or add new columns to the leaderboard

custom_meta = {'team': 'KLSRS',
               'model_type': 'Random_Forest',
               'Member1': 'Krutarth Saraswath',
              'Member2': 'Rishabh Salwan'}

mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values,
                                 custom_metadata = custom_meta)

In [ ]:
image_list = []
    filelist1 = os.listdir(imageband_directory)
    # gets list of all image files in the folder
    
    for fpath in filelist1:
    # loop over all the image names in the given folder
        name = imageband_directory+"/"+fpath
        # this is the complete path name for a given image
        
        if fpath.endswith(("B02.tif","B03.tif","B04.tif")):
        # if the image is in the red, blue or green band execute the following code
            image_array = _load_tif(name)
            # image array is the 120x120 matrix for the image (either red, green or blue)
            image_list.append(image_array)
            # For each image we get a list of 120x120x3 matrices
    
    X = np.stack(image_list, axis = 2)
    # Converts the list into an multidimensional array of shape 120x120x3 i.e. each image has been stacked on top of another
    X = np.expand_dims(X, axis = 0)
    # Adds an extra dimension to convert the above shape in the form 1x20x20x3. Probably for keras model input
    X = np.array(X, dtype=np.float32)
    # Convert all elements to float
    X = X/18581
    # Normalising the data for better ML algorithm performance and reduce bias
    X = X.flatten()

In [69]:
x = np.array(np.random.randn(20,20))
y = []
for i in range(19):
    if i%2==0:
        row = []
        for j in range(19):
            if j%2==0:
                row.append(np.mean(np.array([x[i,j],x[i,j+1],x[i+1,j],x[i+1,j+1]])))
        y.append(row)



[[0.46135109547228803,
  0.08138107291485286,
  0.331260341812162,
  0.3964859066509449,
  -0.07608945484238999,
  0.586523432582649,
  -0.2209685494070516,
  -1.080228637109854,
  0.3504681414070307,
  0.28083355000297555],
 [-0.6900393639364326,
  -0.6980715342188357,
  -0.8235247993126664,
  -0.485206076714196,
  -0.45234411262244695,
  -0.559437768147537,
  -0.11690568874542065,
  -0.5230925499375472,
  0.015328077980149873,
  0.4032142586357082],
 [-0.14364235230151245,
  0.2607279545724965,
  0.39890076822823295,
  0.8163611004399401,
  0.5620493359140522,
  0.04904840323164589,
  0.11679416495607975,
  0.18857366988370897,
  -0.41844660424478797,
  0.2930472760287613],
 [0.5070115155639694,
  0.1560349844093168,
  -0.42960993044445894,
  1.710332790381147,
  -0.8312905418304141,
  -0.056350445362899126,
  0.4574858967936962,
  0.03370299548311942,
  0.45466972950458295,
  -0.36026188007428517],
 [0.028815352406969464,
  0.6339215773930713,
  -0.46484288490553016,
  0.23030855663

In [73]:
np.shape(np.array(y))

(10, 10)

In [ ]:
"""
Rough work Code
"""

In [25]:
filelist1 = os.listdir("competition_data/trainingdata/forest")

In [26]:
filelist1

['S2A_MSIL2A_20171101T094131_64_47',
 'S2A_MSIL2A_20171121T112351_77_8',
 'S2A_MSIL2A_20170701T093031_19_10',
 'S2A_MSIL2A_20180506T100031_32_77',
 'S2A_MSIL2A_20171101T094131_31_77',
 'S2A_MSIL2A_20180506T100031_7_37',
 'S2A_MSIL2A_20170905T095031_54_17',
 'S2A_MSIL2A_20171121T112351_62_70',
 'S2A_MSIL2A_20180205T100211_67_86',
 'S2A_MSIL2A_20171002T112111_63_40',
 'S2A_MSIL2A_20171104T095201_8_43',
 'S2A_MSIL2A_20180506T100031_63_77',
 'S2A_MSIL2A_20171101T094131_68_28',
 'S2A_MSIL2A_20180508T104031_28_1',
 'S2A_MSIL2A_20171221T112501_33_82',
 'S2A_MSIL2A_20170701T093031_12_68',
 'S2A_MSIL2A_20180506T100031_25_34',
 'S2A_MSIL2A_20180526T100031_72_68',
 'S2A_MSIL2A_20170701T093031_50_24',
 'S2A_MSIL2A_20170813T112121_47_14',
 'S2A_MSIL2A_20171208T093351_59_47',
 'S2A_MSIL2A_20171002T112112_38_14',
 'S2A_MSIL2A_20180508T104031_29_13',
 'S2A_MSIL2A_20180413T95032_0_32',
 'S2A_MSIL2A_20171208T093351_81_63',
 'S2A_MSIL2A_20170701T093031_4_71',
 'S2A_MSIL2A_20171221T112501_51_59',
 'S2A_MS

In [29]:
data = "/Users/rishabhsalwan/competition_data/trainingdata/forest/S2A_MSIL2A_20170613T101031_5_70/S2A_MSIL2A_20170613T101031_5_70_B01.tif"

In [32]:
img = tfds.core.lazy_imports.PIL_Image.open(data)
img

In [42]:
image = np.array(img.getdata()).reshape(img.size).astype(np.float32)
image

array([[132., 122., 161., 136., 127., 142., 154., 129., 124., 123., 130.,
        154., 146., 151., 143., 162., 201., 182., 225., 176.],
       [188., 179., 196., 193., 163., 142., 141., 138., 150., 130., 130.,
        179., 169., 151., 136., 163., 184., 179., 259., 182.],
       [242., 310., 249., 251., 295., 182., 144., 160., 138., 126., 152.,
        183., 159., 146., 145., 162., 174., 142., 203., 188.],
       [228., 279., 249., 187., 263., 261., 211., 143., 125., 150., 173.,
        156., 136., 138., 140., 166., 210., 144., 146., 166.],
       [283., 240., 208., 168., 190., 270., 294., 226., 154., 157., 151.,
        138., 150., 142., 138., 171., 224., 188., 165., 164.],
       [262., 266., 207., 202., 168., 189., 263., 243., 200., 213., 160.,
        163., 214., 177., 132., 142., 185., 195., 180., 166.],
       [202., 239., 183., 173., 207., 189., 215., 249., 274., 308., 246.,
        204., 194., 163., 142., 156., 177., 170., 177., 155.],
       [178., 190., 210., 238., 243., 219

In [43]:
X = np.stack(image, axis = 2)
X

AxisError: axis 2 is out of bounds for array of dimension 2

In [104]:
list1 = []
x = np.array(np.random.randn(20,20))
y = np.array(np.random.randn(20,20))
z = np.array(np.random.randn(20,20))
a = np.array(np.random.randn(20,20))
list1.append(x)
list1.append(y)
list1.append(z)
list1.append(a)
list1

[array([[-4.79041617e-02, -4.23594086e-01,  6.68092766e-01,
          2.35724992e+00, -5.42397141e-01, -1.30802854e+00,
          3.13406574e-01, -8.85226411e-01,  6.33061835e-01,
          2.68431667e-01, -1.46565970e+00,  8.97890017e-01,
          1.01438454e+00, -5.49942790e-02,  1.99767932e+00,
          1.52703159e-02, -3.44171605e-01,  1.56310648e+00,
          8.86291142e-01, -1.37665442e+00],
        [ 1.03680848e+00, -7.18540738e-02, -8.64894384e-01,
          2.52598619e-01,  1.18376890e+00, -2.07049029e-01,
          9.09040966e-01,  6.12420083e-01,  8.05981346e-01,
         -1.17398878e+00,  3.44518074e-01, -5.79897137e-01,
         -8.03356520e-01,  2.91990542e-01, -8.11083144e-02,
          1.72219875e+00,  1.10102760e+00,  5.70041120e-02,
          8.72310963e-01,  1.32925788e-01],
        [ 1.11753444e+00, -7.14768881e-01,  7.65992270e-01,
          5.80945863e-01, -5.03593799e-01, -6.18476580e-01,
          6.79101163e-01,  6.35798762e-01,  2.87052612e-01,
         -3.

In [105]:
X = np.stack(list1, axis=2)

In [106]:
X

array([[[-0.04790416,  0.15229416, -1.01851367,  0.3643129 ],
        [-0.42359409,  0.15717043, -0.42540142,  1.12779574],
        [ 0.66809277, -0.3116535 ,  1.03791364,  0.03453115],
        ...,
        [ 1.56310648, -0.4437938 ,  1.59108532,  0.16059068],
        [ 0.88629114,  0.01911173, -0.72591069, -0.83628878],
        [-1.37665442,  1.48205322,  1.73342398,  0.14821479]],

       [[ 1.03680848, -1.70378104,  0.44054245, -0.85305405],
        [-0.07185407, -0.0476158 ,  0.39476949, -0.55153338],
        [-0.86489438, -0.6161087 ,  1.5921178 , -0.22958549],
        ...,
        [ 0.05700411,  0.37772015,  2.19335064, -0.19093182],
        [ 0.87231096, -2.20153298,  0.4672851 ,  0.48123468],
        [ 0.13292579, -1.90134232,  1.26089047,  1.03718848]],

       [[ 1.11753444,  0.5971704 , -0.37238407, -1.67646308],
        [-0.71476888, -0.35404421,  1.28258971,  1.59456309],
        [ 0.76599227,  1.49984137, -0.39086195, -1.00370231],
        ...,
        [ 0.72728846,  0.95

In [108]:
X.shape

(20, 20, 4)

In [102]:
y[0,0]

-0.4188054365206076

In [103]:
z[0,0]

-1.1150210936929819